In [156]:
import pandas as pd
import numpy as np

df_calib = pd.read_excel('/content/각 프레임에 대한 카메라 파라미터_스무딩버전3.xlsx')
df_deepsort = pd.read_excel('/content/video_frame_scaled3.xlsx')

# 객체추적 결과 먼저 전처리

In [157]:
df_deepsort.head()

,Unnamed: 0,frame_idx,frame_time,track_id,number,team,role,x1,y1,x2,...,width,height,x_center,y_center,x1_scaled,x2_scaled,x_center_scaled,y1_scaled,y2_scaled,y_center_scaled
0,0,1,0.018,NaN,NaN,L,Player,1651,850,1679,...,28,75,1665.0,887.5,333.868889,339.531111,336.700000,179.729730,195.588235,187.658983
1,1,1,0.018,NaN,NaN,L,Player,1948,576,1993,...,45,63,1970.5,607.5,393.928889,403.028889,398.478889,121.793323,135.114467,128.453895
2,2,1,0.018,NaN,NaN,NaN,Staff,2014,1179,2053,...,39,79,2033.5,1218.5,407.275556,415.162222,411.218889,249.295707,266.000000,257.647854
3,3,1,0.018,NaN,NaN,L,Player,2081,457,2118,...,37,63,2099.5,488.5,420.824444,428.306667,424.565556,96.631161,109.952305,103.291733
4,4,1,0.018,NaN,NaN,R,Player,1211,539,1248,...,37,55,1229.5,566.5,244.891111,252.373333,248.632222,113.969793,125.599364,119.784579


In [158]:
df_deepsort = df_deepsort.iloc[:, 1:];df_deepsort.head()

,frame_idx,frame_time,track_id,number,team,role,x1,y1,x2,y2,width,height,x_center,y_center,x1_scaled,x2_scaled,x_center_scaled,y1_scaled,y2_scaled,y_center_scaled
0,1,0.018,NaN,NaN,L,Player,1651,850,1679,925,28,75,1665.0,887.5,333.868889,339.531111,336.700000,179.729730,195.588235,187.658983
1,1,0.018,NaN,NaN,L,Player,1948,576,1993,639,45,63,1970.5,607.5,393.928889,403.028889,398.478889,121.793323,135.114467,128.453895
2,1,0.018,NaN,NaN,NaN,Staff,2014,1179,2053,1258,39,79,2033.5,1218.5,407.275556,415.162222,411.218889,249.295707,266.000000,257.647854
3,1,0.018,NaN,NaN,L,Player,2081,457,2118,520,37,63,2099.5,488.5,420.824444,428.306667,424.565556,96.631161,109.952305,103.291733
4,1,0.018,NaN,NaN,R,Player,1211,539,1248,594,37,55,1229.5,566.5,244.891111,252.373333,248.632222,113.969793,125.599364,119.784579


In [159]:
# 프레임별 bbox 정보와 역할/팀 정보를 저장할 딕셔너리
frame_bbox_dfs = {}
frame_role_dfs = {}

# 고유한 프레임 인덱스 순회
for frame_id in df_deepsort['frame_idx'].unique():
    frame_row = df_deepsort[df_deepsort['frame_idx'] == frame_id]
    frame_id = int(frame_id)
    # bbox 정보 (x_center_scaled, y1_scaled)
    frame_bbox_dfs[frame_id] = frame_row[['x_center_scaled', 'y1_scaled']].reset_index(drop=True)

    # 역할/팀 정보 (role, team)
    frame_role_dfs[frame_id] = frame_row[['role', 'team', 'number']].reset_index(drop=True)


In [160]:
frame_role_dfs

{1:           role team  number
 0       Player    L     NaN
 1       Player    L     NaN
 2        Staff  NaN     NaN
 3       Player    L     NaN
 4       Player    R     NaN
 5       Player    R     NaN
 6       Player    L     NaN
 7       Player    R     NaN
 8       Player    L     NaN
 9        Staff  NaN     NaN
 10      Player    R     NaN
 11      Player    R     NaN
 12      Player    L     NaN
 13    Main Ref  NaN     NaN
 14      Player    L     NaN
 15      Player    R     NaN
 16      Player    L     NaN
 17      Player    L     NaN
 18      Player    R     NaN
 19      Player    R     NaN
 20  Goalkeeper  NaN     NaN
 21      Player    L     NaN
 22    Side Ref  NaN     NaN
 23    Side Ref  NaN     NaN
 24      Player    R     NaN
 25      Player    R     NaN,
 2:           role team  number
 0       Player    L     NaN
 1       Player    L     NaN
 2       Player    R     NaN
 3       Player    L     NaN
 4        Staff  NaN     NaN
 5       Player    L     NaN
 6     

# 호모그래피 행렬 프레임별로 빼내기

In [161]:
df_calib.head()

,image_id,pan_degrees,tilt_degrees,roll_degrees,position_meters,aov_radian,aov_degrees,x_focal_length,y_focal_length,principal_point,...,mask_lines,mask_circles,loss_ndc_lines_distances_raw,loss_ndc_lines,loss_ndc_circles_distances_raw,loss_ndc_circles,loss_ndc_total,points_line,points_circle,homography_smoothed
0,0,-9.678379,67.367714,-0.190248,"[-1.9619878108195623, 95.78398781224168, -37.4...",0.608768,34.413307,734.631775,734.631775,"[227.5, 128.0]",...,"[[[True, True, True, True], [True, True, True,...","[[[False, True, False, True, False, True, True...","[[[0.0010628722375258803, 0.004536682739853859...",0.009984,"[[[0.0, 0.0069590783677995205, 0.0, 0.00815042...",0.006658,0.016642,"tensor([[[ 45.8989, 1.9956, 59.8681, 34.92...","tensor([[[ 0.0000, 324.2857, 0.0000, 315.30...",[[ 1.34910608e-03 4.66266831e-05 -4.85658360e...
1,1,-9.741721,67.354617,-0.169291,"[-1.9327309162802406, 95.77892350155223, -37.5...",0.608927,34.592983,730.575928,730.575928,"[227.5, 128.0]",...,"[[[True, True, True, True], [True, True, True,...","[[[False, True, True, False, True, False, Fals...","[[[0.004822007846087217, 0.0007496190373785794...",0.009008,"[[[0.0, 0.006156992167234421, 0.00187130249105...",0.006107,0.015115,"tensor([[[ 48.8923, 39.9121, 2.9934, 59.86...","tensor([[[ 0.0000, 351.2264, 303.3319, 0.00...",[[ 1.35560714e-03 3.48780629e-05 -4.87870923e...
2,2,-9.800128,67.342903,-0.152592,"[-1.910646425857662, 95.7732146229886, -37.533...",0.609079,35.889168,702.484131,702.484131,"[227.5, 128.0]",...,"[[[True, True, True, True], [True, True, True,...","[[[False, True, False, True, True, True, False...","[[[0.0036418067757040262, 0.002915193559601903...",0.010975,"[[[0.0, 0.0086074722930789, 0.0, 0.00775554869...",0.004037,0.015012,"tensor([[[ 12.9714, 24.9451, 59.8681, 1.99...","tensor([[[ 0.0000, 345.2396, 0.0000, 295.34...",[[ 1.36107952e-03 2.58134025e-05 -4.89888379e...
3,3,-9.853953,67.332516,-0.139835,"[-1.8952036050684673, 95.7666158002501, -37.54...",0.609226,34.558025,731.361877,731.361877,"[227.5, 128.0]",...,"[[[True, True, True, True], [True, True, True,...","[[[False, False, True, True, True, False, True...","[[[0.004688375163823366, 0.003247351385653019,...",0.011413,"[[[0.0, 0.0, 0.0002990099892485887, 0.00285889...",0.003025,0.014438,"tensor([[[ 50.8879, 37.9165, 1.9956, 61.86...","tensor([[[ 0.0000, 0.0000, 344.2418, 294.35...",[[ 1.36552321e-03 1.94327019e-05 -4.91710727e...
4,4,-9.903546,67.323405,-0.130705,"[-1.8858717194292975, 95.75888165703596, -37.5...",0.609369,34.460110,733.571411,733.571411,"[227.5, 128.0]",...,"[[[True, True, True, True], [True, True, True,...","[[[True, False, False, True, False, True, Fals...","[[[0.011104949750006199, 0.008681845851242542,...",0.010477,"[[[0.0038223799783736467, 0.0, 0.0, 0.00692903...",0.006956,0.017433,"tensor([[[ 19.9560, 35.9209, 60.8659, 3.99...","tensor([[[297.3451, 0.0000, 0.0000, 347.23...",[[ 1.36893822e-03 1.57359610e-05 -4.93337969e...


In [162]:
frame_homography_dfs = {}
for i in df_calib['image_id']:
  row = df_calib[df_calib['image_id'] == i]
  frame_homography_dfs[i] = row['homography']

In [163]:
def convert_with_homography(points_2d, H):
    """
    points_2d: (N, 2) 형태의 numpy 배열 (x, y)
    H: (3, 3) 호모그래피 행렬
    반환: (N, 2) 형태의 3D 평면상의 점 (X, Y)
    """
    points_2d_homo = np.hstack([points_2d, np.ones((len(points_2d), 1))])  # -> (x, y, 1)
    points_3d_homo = (H @ points_2d_homo.T).T  # H로 변환 (from image to world)
    points_3d = points_3d_homo[:, :2] / points_3d_homo[:, 2][:, np.newaxis]
    return points_3d

# 시각화 합쳐보기

In [164]:
# [frame_homography_dfs] 각 프레임별로 호모그래피 행렬 존재
# [frame_bbox_dfs] 각 프레임별 선수들 bbox 존재
# [frame_role_dfs] 각 프레임별 선수들 역할 존재

In [165]:
import ast
import numpy as np
import pandas as pd

def extract_3x3_homography(h_raw):
    """
    문자열로 저장된 homography 값을 안전하게 파싱해 3x3 numpy 행렬로 변환
    """
    if isinstance(h_raw, pd.Series):
        h_raw = h_raw.iloc[0]

    if isinstance(h_raw, str):
        try:
            parsed = ast.literal_eval(h_raw)
            return np.array(parsed)
        except Exception as e:
            raise ValueError(f"Failed to parse homography string: {e}")

    raise TypeError(f"Unsupported type: {type(h_raw)}")


In [166]:
# 스무딩 H 행렬 쓸 때는 이 코드를 써야함.

def extract_3x3_homography_smoothing(H):
    # 줄바꿈 제거
    H_raw = H.iloc[0]
    h_str = H_raw.replace('\n', ' ')

    # 리스트 사이 공백 → 쉼표 추가
    h_str = re.sub(r'\]\s+\[', '], [', h_str)

    # 숫자 사이 공백 → 쉼표 추가
    h_str_fixed = re.sub(r'(?<=\d)\s+(?=-?\d)', ', ', h_str)

    try:
        arr = eval(h_str_fixed)
        arr_np = np.array(arr)
        if arr_np.shape == (3, 3):
            return arr_np
        else:
            raise ValueError("❗결과가 3x3 행렬이 아닙니다.")
    except Exception as e:
        raise ValueError(f"🛑 파싱 실패: {e}\n입력 문자열: {h_str_fixed}")

In [167]:
# 먼저 각 프레임별 선수들 위치를 호모그래피 행렬로 변환해주기
# 추가로 각 중계 영상 꼭짓점도 변환해주기

frame_transformed_dfs = {}
frame_corners_dfs = {}

points_corners = [[0, 0], [455, 0], [455, 256], [0, 256]]

# 두 딕셔너리의 공통된 프레임만 사용
common_keys = set(frame_bbox_dfs.keys()) & set(frame_homography_dfs.keys())

for frame_id in common_keys:
    # 좌표 가져오기
    coords_df = frame_bbox_dfs[frame_id][['x_center_scaled', 'y1_scaled']].copy()

    # 호모그래피 행렬 가져오기
    # 스무딩 버전 쓸 때는 함수 다른 거 써야함

    H = extract_3x3_homography_smoothing(frame_homography_dfs[frame_id])

    # 호모그래피 변환
    transformed = convert_with_homography(coords_df.values, H)
    transformed_corner = convert_with_homography(np.array(points_corners), H)

    # 결과 저장 (x', y'만)
    transformed_df = pd.DataFrame(transformed[:, :2], columns=['x_mapped', 'y_mapped'])
    frame_transformed_dfs[frame_id] = transformed_df

    frame_corners_dfs[frame_id] = transformed_corner


In [168]:
# frame_transformed_dfs 길이에 막제 frame_role_dfs 길이 변환
# 만약 프레임 길이가 같다면 이 코드를 실행해도 큰 변화가 없을 것

# 공통된 frame_id만 추출
valid_frame_ids = frame_transformed_dfs.keys()

# frame_role_dfs에서 해당되는 프레임만 남김
frame_role_dfs = {fid: frame_role_dfs[fid] for fid in valid_frame_ids if fid in frame_role_dfs}


# 본격적인 시각화 (동영상으로 바꿀 때는 실행 안 해도 됨)

In [48]:
# 우리가 사용할 것
# frame_corners_dfs, frame_transformed_dfs, frame_role_dfs

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# 팀별 색
team_colors = {'L': 'red', 'R': 'green'}

# 축구장 라인
field_lines = [
    ([-52.5,  20.16, 0.], [-36.0,  20.16, 0.]),
    ([-52.5, -20.16, 0.], [-36.0, -20.16, 0.]),
    ([-36.0, -20.16, 0.], [-36.0,  20.16, 0.]),
    ([36.0, 20.16, 0.], [52.5, 20.16, 0.]),
    ([36.0, -20.16, 0.], [52.5, -20.16, 0.]),
    ([36.0, -20.16, 0.], [36.0, 20.16, 0.]),
    ([-52.5, 9.16, 0.], [-47.0, 9.16, 0.]),
    ([-52.5, -9.16, 0.], [-47.0, -9.16, 0.]),
    ([-47.0, -9.16, 0.], [-47.0, 9.16, 0.]),
    ([47.0, 9.16, 0.], [52.5, 9.16, 0.]),
    ([47.0, -9.16, 0.], [52.5, -9.16, 0.]),
    ([47.0, -9.16, 0.], [47.0, 9.16, 0.]),
    ([-52.5, -34.0, 0.], [52.5, -34.0, 0.]),
    ([-52.5, 34.0, 0.], [52.5, 34.0, 0.]),
    ([-52.5, -34.0, 0.], [-52.5, 34.0, 0.]),
    ([52.5, -34.0, 0.], [52.5, 34.0, 0.]),
    ([0.0, -34.0, 0.], [0.0, 34.0, 0.]),
    ([-52.5, -3.66, -2.44], [-52.5, 3.66, -2.44]),
    ([-52.5, 3.66, -2.44], [-52.5, 3.66, 0.]),
    ([-52.5, -3.66, -2.44], [-52.5, -3.66, 0.]),
    ([52.5, -3.66, -2.44], [52.5, 3.66, -2.44]),
    ([52.5, -3.66, -2.44], [52.5, -3.66, 0.]),
    ([52.5, 3.66, -2.44], [52.5, 3.66, 0.]),
    ([-36.0, -7.31, 0.], [-36.0, 7.31, 0.]),
    ([36.0, -7.31, 0.], [36.0, 7.31, 0.]),
]

def draw_line(ax, start, end, color='green', lw=2):
    xs, ys, zs = zip(start, end)
    zs = [-z for z in zs]  # z축 반전
    ax.plot(xs, ys, zs, color=color, lw=lw)

# 프레임별 시각화
for frame_id in sorted(frame_transformed_dfs.keys()):
    if frame_id not in frame_corners_dfs or frame_id not in frame_role_dfs:
        continue  # skip 누락된 프레임

    bbox_2d = frame_transformed_dfs[frame_id].values  # shape: (N, 2)
    bbox_3d = np.column_stack((bbox_2d, np.zeros(len(bbox_2d))))  # z = 0
    bbox_3d[:, 1] *= -1  # y값만 반전

    corners_2d = frame_corners_dfs[frame_id]  # shape: (4, 2) 또는 (M, 2)
    keypoints_xyz_corners = np.column_stack((corners_2d, np.zeros(len(corners_2d))))  # z = 0
    keypoints_xyz_corners[:, 1] *= -1 # y 값만 반전

    roles = frame_role_dfs[frame_id]['role'].values
    teams = frame_role_dfs[frame_id]['team'].values

    fig = plt.figure(figsize=(14, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_title(f"3D Soccer Field - Frame {frame_id}", fontsize=14)

    # 축구장 라인 그리기
    for start, end in field_lines:
        draw_line(ax, start, end)

    # 키포인트 (코너) 시각화
    ax.scatter(
        keypoints_xyz_corners[:, 0],
        keypoints_xyz_corners[:, 1],
        -keypoints_xyz_corners[:, 2],
        color='blue',
        s=10,
        label='Corners'
    )

    # 이미 라벨 추가한 팀들을 저장
    shown_labels = set()

    # 선수 시각화
    for i in range(len(bbox_3d)):
        if roles[i].strip().lower() == 'player':
            team = teams[i]
            color = team_colors.get(team, 'gray')

            # 팀 이름으로 라벨 한 번만 추가
            label = f'Player-{team}' if team not in shown_labels else None
            shown_labels.add(team)

            ax.scatter(
                bbox_3d[i, 0],
                bbox_3d[i, 1],
                -bbox_3d[i, 2],
                color=color,
                s=10,
                label=label
            )
    # 코너 4개로 면 만들기 (가능한 경우만)
    if len(keypoints_xyz_corners) >= 4:
        corner_points = keypoints_xyz_corners[:4].copy()
        corner_points[:, 2] = -corner_points[:, 2]
        verts = [corner_points]
        poly = Poly3DCollection(verts, alpha=0.3, facecolor='skyblue', edgecolor='blue')
        ax.add_collection3d(poly)

    # 축 설정
    ax.set_xlabel("X (width)")
    ax.set_ylabel("Y (height)")
    ax.set_zlabel("Z (up/down)")
    ax.set_xlim(-60, 60)
    ax.set_ylim(-40, 40)
    ax.set_zlim(-10, 10)
    ax.legend()

    plt.show()


# 결과 동영상으로 변환하기

In [169]:
import os

# 저장 경로 생성
save_dir = "frames_3d"
os.makedirs(save_dir, exist_ok=True)


# 팀별 색
team_colors = {'L': 'blue', 'R': 'red'}

# 축구장 라인
field_lines = [
    ([-52.5,  20.16, 0.], [-36.0,  20.16, 0.]),
    ([-52.5, -20.16, 0.], [-36.0, -20.16, 0.]),
    ([-36.0, -20.16, 0.], [-36.0,  20.16, 0.]),
    ([36.0, 20.16, 0.], [52.5, 20.16, 0.]),
    ([36.0, -20.16, 0.], [52.5, -20.16, 0.]),
    ([36.0, -20.16, 0.], [36.0, 20.16, 0.]),
    ([-52.5, 9.16, 0.], [-47.0, 9.16, 0.]),
    ([-52.5, -9.16, 0.], [-47.0, -9.16, 0.]),
    ([-47.0, -9.16, 0.], [-47.0, 9.16, 0.]),
    ([47.0, 9.16, 0.], [52.5, 9.16, 0.]),
    ([47.0, -9.16, 0.], [52.5, -9.16, 0.]),
    ([47.0, -9.16, 0.], [47.0, 9.16, 0.]),
    ([-52.5, -34.0, 0.], [52.5, -34.0, 0.]),
    ([-52.5, 34.0, 0.], [52.5, 34.0, 0.]),
    ([-52.5, -34.0, 0.], [-52.5, 34.0, 0.]),
    ([52.5, -34.0, 0.], [52.5, 34.0, 0.]),
    ([0.0, -34.0, 0.], [0.0, 34.0, 0.]),
    ([-52.5, -3.66, -2.44], [-52.5, 3.66, -2.44]),
    ([-52.5, 3.66, -2.44], [-52.5, 3.66, 0.]),
    ([-52.5, -3.66, -2.44], [-52.5, -3.66, 0.]),
    ([52.5, -3.66, -2.44], [52.5, 3.66, -2.44]),
    ([52.5, -3.66, -2.44], [52.5, -3.66, 0.]),
    ([52.5, 3.66, -2.44], [52.5, 3.66, 0.]),
    ([-36.0, -7.31, 0.], [-36.0, 7.31, 0.]),
    ([36.0, -7.31, 0.], [36.0, 7.31, 0.]),
]

def draw_line(ax, start, end, color='green', lw=2):
    xs, ys, zs = zip(start, end)
    zs = [-z for z in zs]  # z축 반전
    ax.plot(xs, ys, zs, color=color, lw=lw)

# 프레임별 시각화
for frame_id in sorted(frame_transformed_dfs.keys()):
    if frame_id not in frame_corners_dfs or frame_id not in frame_role_dfs:
        continue  # skip 누락된 프레임

    bbox_2d = frame_transformed_dfs[frame_id].values  # shape: (N, 2)
    bbox_3d = np.column_stack((bbox_2d, np.zeros(len(bbox_2d))))  # z = 0
    bbox_3d[:, 1] *= -1  # y값만 반전

    corners_2d = frame_corners_dfs[frame_id]  # shape: (4, 2) 또는 (M, 2)
    keypoints_xyz_corners = np.column_stack((corners_2d, np.zeros(len(corners_2d))))  # z = 0
    keypoints_xyz_corners[:, 1] *= -1 # y 값만 반전

    roles = frame_role_dfs[frame_id]['role'].values
    teams = frame_role_dfs[frame_id]['team'].values

    fig = plt.figure(figsize=(14, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_title(f"3D Soccer Field - Frame {frame_id}", fontsize=14)

    # 축구장 라인 그리기
    for start, end in field_lines:
        draw_line(ax, start, end)

    # # 키포인트 (코너) 시각화
    # ax.scatter(
    #     keypoints_xyz_corners[:, 0],
    #     keypoints_xyz_corners[:, 1],
    #     -keypoints_xyz_corners[:, 2],
    #     color='black',
    #     s=10,
    #     label='Corners'
    # )

    # 이미 라벨 추가한 팀들을 저장
    shown_labels = set()

    # 선수 시각화
    for i in range(len(bbox_3d)):
        if roles[i].strip().lower() == 'player':
            team = teams[i]
            color = team_colors.get(team, 'gray')

            # 팀 이름으로 라벨 한 번만 추가
            label = f'Player-{team}' if team not in shown_labels else None
            shown_labels.add(team)

            ax.scatter(
                bbox_3d[i, 0],
                bbox_3d[i, 1],
                -bbox_3d[i, 2],
                color=color,
                s=10,
                label=label
            )
    # 코너 4개로 면 만들기 (가능한 경우만)
    if len(keypoints_xyz_corners) >= 4:
        corner_points = keypoints_xyz_corners[:4].copy()
        corner_points[:, 2] = -corner_points[:, 2]
        verts = [corner_points]
        poly = Poly3DCollection(verts, alpha=0.3, facecolor='skyblue', edgecolor='blue')
        ax.add_collection3d(poly)

    # 축 설정
    ax.set_xlabel("X (width)")
    ax.set_ylabel("Y (height)")
    ax.set_zlabel("Z (up/down)")
    ax.set_xlim(-60, 60)
    ax.set_ylim(-40, 40)
    ax.set_zlim(-10, 10)
    ax.legend()


    # 저장 파일 이름
    save_path = os.path.join(save_dir, f"frame_{frame_id:04d}.png")
    plt.savefig(save_path)
    plt.close(fig)  # 메모리 절약

In [170]:
import imageio.v2 as imageio
import os

image_dir = "frames_3d"
output_path = "soccer_3d_스무딩3.mp4"
fps = 60

# 프레임 파일 정렬
frame_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.png')])
images = []

for file in frame_files:
    filepath = os.path.join(image_dir, file)
    images.append(imageio.imread(filepath))

# 동영상 저장
imageio.mimsave(output_path, images, fps=fps)


# 영상 여러개 이어 붙이는 코드

In [175]:
import cv2
# 이어붙일 영상 목록
video_paths = ["/content/soccer_3d_스무딩1.mp4", "/content/soccer_3d_스무딩2.mp4", "/content/soccer_3d_스무딩3.mp4"]

# 첫 번째 영상에서 기본 속성 가져오기
cap0 = cv2.VideoCapture(video_paths[0])
fps = cap0.get(cv2.CAP_PROP_FPS)
width = int(cap0.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap0.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap0.release()

# 출력 비디오 객체 생성
out = cv2.VideoWriter("concatenated_video_스무딩.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

for path in video_paths:
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        print(f"❌ {path} 열기 실패!")
        continue

    print(f"📼 {path} 처리 중...")

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    cap.release()

out.release()
print("✅ 영상 이어붙이기 완료!")


📼 /content/soccer_3d_스무딩1.mp4 처리 중...
📼 /content/soccer_3d_스무딩2.mp4 처리 중...
📼 /content/soccer_3d_스무딩3.mp4 처리 중...
✅ 영상 이어붙이기 완료!


In [176]:
import cv2
import numpy as np

def concat_videos_horizontally(video_path1, video_path2, output_path):
    cap1 = cv2.VideoCapture(video_path1)
    cap2 = cv2.VideoCapture(video_path2)

    if not cap1.isOpened() or not cap2.isOpened():
        print("둘 중 하나의 비디오를 열 수 없습니다.")
        return

    # 두 비디오의 속성
    fps = cap1.get(cv2.CAP_PROP_FPS)
    width1 = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
    height1 = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width2 = int(cap2.get(cv2.CAP_PROP_FRAME_WIDTH))
    height2 = int(cap2.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 세로 높이를 맞추기 위해 작은 쪽에 resize 적용할 예정
    final_height = max(height1, height2)
    final_width = width1 + width2

    # 결과 비디오 저장
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (final_width, final_height))

    while True:
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()

        if not ret1 or not ret2:
            break

        # 사이즈 맞춰주기
        if frame1.shape[0] != final_height:
            frame1 = cv2.resize(frame1, (width1, final_height))
        if frame2.shape[0] != final_height:
            frame2 = cv2.resize(frame2, (width2, final_height))

        # 가로로 붙이기
        combined = np.hstack((frame1, frame2))
        out.write(combined)

    cap1.release()
    cap2.release()
    out.release()
    print("🎉 영상 합치기 완료:", output_path)


In [ ]:
concat_videos_horizontally("/content/final_sample_calib.mov", "/content/concatenated_video_스무딩.mp4", "output.mp4")